## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Rikitea,-23.1203,-134.9692,78.22,72,11,14.05,PF,2021-02-05 23:26:20,light rain
1,1,Horsham,51.0640,-0.3272,44.60,87,75,4.61,GB,2021-02-05 23:27:09,broken clouds
2,2,Longyearbyen,78.2186,15.6401,24.80,86,75,9.22,SJ,2021-02-05 23:25:13,light snow
3,3,Flinders,-34.5833,150.8552,70.00,91,100,8.99,AU,2021-02-05 23:25:13,light rain
4,4,Punta Arenas,-53.1500,-70.9167,53.60,47,0,26.46,CL,2021-02-05 23:27:09,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Rikitea,-23.1203,-134.9692,78.22,72,11,14.05,PF,2021-02-05 23:26:20,light rain
5,5,Hilo,19.7297,-155.0900,78.80,57,1,11.50,US,2021-02-05 23:13:13,light rain
6,6,Tecoanapa,16.5167,-98.7500,81.63,65,24,7.81,MX,2021-02-05 23:27:10,few clouds
10,10,Mahebourg,-20.4081,57.7000,77.00,83,75,3.44,MU,2021-02-05 23:23:22,light rain
11,11,Vaini,-21.2000,-175.2000,84.20,84,75,6.91,TO,2021-02-05 23:23:17,broken clouds
17,17,Banjar,-8.1900,114.9675,78.80,89,40,4.61,ID,2021-02-05 23:27:11,light rain
18,18,Nadadores,27.0500,-101.6000,80.01,20,0,5.01,MX,2021-02-05 23:27:12,clear sky
22,22,Vaitape,-16.5167,-151.7500,81.55,63,1,3.58,PF,2021-02-05 23:27:12,clear sky
24,24,Tera,14.0078,0.7531,77.67,17,20,9.91,NE,2021-02-05 23:27:13,few clouds
25,25,Mahibadhoo,3.7833,72.9667,81.10,71,87,9.46,MV,2021-02-05 23:23:14,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.22,light rain,-23.1203,-134.9692,
5,Hilo,US,78.80,light rain,19.7297,-155.0900,
6,Tecoanapa,MX,81.63,few clouds,16.5167,-98.7500,
10,Mahebourg,MU,77.00,light rain,-20.4081,57.7000,
11,Vaini,TO,84.20,broken clouds,-21.2000,-175.2000,
17,Banjar,ID,78.80,light rain,-8.1900,114.9675,
18,Nadadores,MX,80.01,clear sky,27.0500,-101.6000,
22,Vaitape,PF,81.55,clear sky,-16.5167,-151.7500,
24,Tera,NE,77.67,few clouds,14.0078,0.7531,
25,Mahibadhoo,MV,81.10,overcast clouds,3.7833,72.9667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df = hotel_df.dropna()
hotel_df=hotel_df[hotel_df["Hotel Name"]!= ""]
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.22,light rain,-23.1203,-134.9692,Pension Maro'i
5,Hilo,US,78.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Tecoanapa,MX,81.63,few clouds,16.5167,-98.7500,Hotel Manglares
10,Mahebourg,MU,77.00,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
11,Vaini,TO,84.20,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
672,Coahuayana,MX,75.20,scattered clouds,18.7333,-103.6833,Hotel Los Arcos
674,Arawa,PG,84.61,few clouds,-6.2132,155.5367,Coastline Guest House
679,Cabedelo,BR,78.80,few clouds,-6.9811,-34.8339,Hotel Pigalle
694,Gibara,CU,78.80,scattered clouds,21.1097,-76.1317,Sol y Mar


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))